Specifications:

1. Take dataframe containing content, title columns
2. Output dataframe appended with new column called country

Exceptions:

1. If dataframe does not have content or title then output error with print warning
2. If articles do not contain geopolitical entities, then it outputs NA

Structure:
    
1. Machine learning model is saved outside 


Import Modules

In [23]:
import pandas as pd
import numpy as np
import spacy
from sklearn.externals import joblib

/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Create a Dataframe for Machine Learning Model

article_list = original data
df = data frame to be used for machine learning
row = each geopolitical entity
column = features to be tested for importance in statistical model

In [3]:
state_country = pd.read_csv("/Users/jihunlee/Desktop/state.csv", encoding='latin-1')
country_codes = pd.read_csv("/Users/jihunlee/Desktop/country-codes.csv", encoding='latin-1')
city_codes1 = pd.read_csv("/Users/jihunlee/Desktop/city1.csv", encoding='latin-1')
city_codes2 = pd.read_csv("/Users/jihunlee/Desktop/city2.csv", encoding='latin-1')
city_codes3 = pd.read_csv("/Users/jihunlee/Desktop/city3.csv", encoding='latin-1')
city_codes = pd.concat([city_codes1, city_codes2, city_codes3])
state_country = state_country.set_index('State').T.to_dict('list')
city_country = city_codes.set_index('city').T.to_dict('list')
country_codes = country_codes.set_index('abbreviation').T.to_dict('list')

/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  import sys
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  if __name__ == '__main__':


In [4]:
def us_or_uk(abbrev):
    if abbrev in ['US','USA','United States']:
        return('United States of America')
    elif abbrev in ['UK']:
        return('United Kingdom')
    else:
        return(abbrev)

In [5]:
def convert_to_country(location):
    if location != None: # location could be None
        state = state_country.get(location) # map state to country
        city = city_country.get(location) # map city to country
        if state != None: # if location can be found in state
            state = [x for x in state if str(x) !='nan'][0] # get rid of nan's and choose the first element
            country = country_codes.get(''.join(state))
        elif city != None:
            city = [x for x in city if str(x) not in ['nan','!']][0] # get rid of nan's and choose the first element
            country = country_codes.get(''.join(city))
        else:
            country = us_or_uk(location)
        return(''.join(country))
    else: 
        return(None)

In [10]:
def convert_to_country2(location):
    if location != None: # location could be None
        state = state_country.get(location) # map state to country
        city = city_country.get(location) # map city to country
        if state != None: # if location can be found in state
            state = [x for x in state if str(x) !='nan'][0] # get rid of nan's and choose the first element
            country = country_codes.get(''.join(state))
        elif city != None:
            city = [x for x in city if str(x) not in ['nan','!']][0] # get rid of nan's and choose the first element
            country = country_codes.get(''.join(city))
        else:
            country = us_or_uk(location)
        return(''.join(country))
    else: 
        return(None)

In [15]:
def in_title(location, title):
    if location in title:
        return(1)
    else:
        return(0)

In [8]:
# Create an exmaple DataFrame 
article = pd.DataFrame({'Title':['China is Attacked'], 
                        'Content':['Japan attacks Beijing with airstrike and kills 500 people.']}) 

In [13]:
def ner_dataframe2(article):
    
    nlp = spacy.load("en_core_web_sm")
    
    # article has Content column and Country column
    content = article.Content
    title = article.Title
    result = pd.DataFrame()
    doc = nlp(str(content))

    # create an array of all entities
    a = [ent for ent in doc.ents]
    if sum([ent.label_ == 'GPE' for ent in a]) > 0:

        a = [str(ent) for ent in doc.ents]

        # create an array of all entity labels
        b = [str(ent.label_) for ent in doc.ents]

        # create an array of all entity positions
        p = [int(ent.start_char) for ent in doc.ents]

        # create a dataframe out of these two 
        temp = pd.DataFrame({'entity':a,'labels':b,'position':p})

        # subset rows whose labels are geopolitical entities
        temp = temp[temp.labels == 'GPE']
        temp = temp.drop(columns = ['labels'])
        temp['entity'] = list(map(convert_to_country2, temp.entity)) # convert all cities and states into countries

        # Group rows by entities, get counts, and get the first of each location count (creating three columns: entity, count, and first location)
        temp1 = temp.groupby(['entity']).count().reset_index().rename(columns={'position':'freq'})
        temp2 = temp.groupby(['entity'])['position'].first().reset_index()
        # Join these two tables to produce a table containing entities, count, and location
        df = temp1.merge(temp2, on = 'entity', how = 'inner')

        # d is a variable measuring the proportion of this frequency
        # higher d means it was mentioned more frequently in the article
        d = df.freq
        all_count = sum(d)
        d = [value/all_count for value in d]

        # e is an array of start character position
        # f is an array of binary values whose 1 is if it is the first geopolitical entity mentioned, and 0 if not
        m = len(df)
        f = [0]*m
        min_index = np.argmin(df.position)
        f[min_index] = 1

        # g is an array of binary values whose 1 is if it is the last geopolitical entity mentioned, and 0 if not
        g = [0]*m
        max_index = np.argmax(df.position)
        g[max_index] = 1

        # h is an array of binary values whose 1 is if it is the most frequently mentioned entity, and 0 if not
        h = [0]*m
        most_index = np.argmax(df.count)
        h[most_index] = 1

        # x is an array of binary values whose 1 is if entity is also in the title, else 0
        x = [in_title(x, title) for x  in df.entity]

        # append columns
        df['proportion'] = d
        df['first'] = f
        df['last'] = g
        df['mode'] = h
        df['title'] = x

        # drop the position column
        df = df.drop(columns=['position'])
    else: 
        df = pd.DataFrame(np.array([None, None, None, None, None, None, None]).reshape(-1,7),
                         columns=['entity','freq','proportion','first','last','mode','title'])
    return(df)

In [16]:
ner_dataframe2(article)

,entity,freq,proportion,first,last,mode,title
0,China,1,0.5,0,1,1,0
1,Japan,1,0.5,1,0,0,0


In [20]:
def sklearn_dataframe(df):
    df = df[['proportion','first','last','mode','title']]
    df = df.astype(float)
    return(df)

In [25]:
df2 = sklearn_dataframe(ner_dataframe2(article))

/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return bound(*args, **kwds)
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [53]:
model = joblib.load('/Users/jihunlee/Desktop/rf_location_extraction.pkl')

In [66]:
country_codes = pd.read_csv("/Users/jihunlee/Desktop/country-codes.csv", encoding='latin-1')
country_codes2 = country_codes.set_index('country').T.to_dict('list')

In [55]:
predictions = model.predict_proba(df2)

In [68]:
predictions

array([[0.94658861, 0.05341139],
       [0.98280003, 0.01719997]])

In [83]:
location_index = np.argmax(predictions[:,0])

In [59]:
df = ner_dataframe2(article)

/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return bound(*args, **kwds)
/Users/jihunlee/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [85]:
def location_extraction(model, article):
    model = joblib.load(model)
    df = ner_dataframe2(article)
    df2 = sklearn_dataframe(df)
    predictions = model.predict_proba(df2)
    location_index = location_index = np.argmax(predictions[:,0])
    location = df.entity[location_index]
    country_codes2.get(''.join(location))
    print(location)

In [ ]:
location_extra

# Training 

In [31]:
def ner_dataframe(article_list):
    
    nlp = spacy.load("en_core_web_sm")
    
    # article_list has content column, Country column
    content_list = article_list.Content
    title_list = article_list.Title
    location_list = article_list.Country
    n = len(article_list)
    result = pd.DataFrame()
    
    for i in range(n):
        
        title = title_list[i]
        content = content_list[i]
        correct_location = location_list[i]
        doc = nlp(content)
        
        # create an array of all entities
        a = [ent for ent in doc.ents]
        if sum([ent.label_ == 'GPE' for ent in a]) > 0:
            
            a = [str(ent) for ent in doc.ents]
            
            # create an array of all entity labels
            b = [str(ent.label_) for ent in doc.ents]

            # create an array of all entity positions
            p = [int(ent.start_char) for ent in doc.ents]

            # create a dataframe out of these two 
            temp = pd.DataFrame({'entity':a,'labels':b,'position':p})

            # subset rows whose labels are geopolitical entities
            temp = temp[temp.labels == 'GPE']
            temp = temp.drop(columns = ['labels'])
            temp['entity'] = list(map(convert_to_country, temp.entity)) # convert all cities and states into countries

            # Group rows by entities, get counts, and get the first of each location count (creating three columns: entity, count, and first location)
            temp1 = temp.groupby(['entity']).count().reset_index().rename(columns={'position':'freq'})
            temp2 = temp.groupby(['entity'])['position'].first().reset_index()
            # Join these two tables to produce a table containing entities, count, and location
            df = temp1.merge(temp2, on = 'entity', how = 'inner')

            # c is the response variable: 1 if this geopolitical entity is the correct location, 0 if not
            c = [location == correct_location.title() for location in df.entity] # .title() for capitalizing the first letter

            # append the column
            df['response'] = c

            # d is a variable measuring the proportion of this frequency
            # higher d means it was mentioned more frequently in the article
            d = df.freq
            all_count = sum(d)
            d = [value/all_count for value in d]

            # e is an array of start character position
            # f is an array of binary values whose 1 is if it is the first geopolitical entity mentioned, and 0 if not
            m = len(df)
            f = [0]*m
            min_index = np.argmin(df.position)
            f[min_index] = 1

            # g is an array of binary values whose 1 is if it is the last geopolitical entity mentioned, and 0 if not
            g = [0]*m
            max_index = np.argmax(df.position)
            g[max_index] = 1

            # h is an array of binary values whose 1 is if it is the most frequently mentioned entity, and 0 if not
            h = [0]*m
            most_index = np.argmax(df.count)
            h[most_index] = 1

            # x is an array of binary values whose 1 is if entity is also in the title, else 0
            x = [in_title(x, title) for x  in df.entity]

            # append columns
            df['proportion'] = d
            df['first'] = f
            df['last'] = g
            df['mode'] = h
            df['title'] = x

            # drop the position column
            df = df.drop(columns=['position'])
        else: 
            df = pd.DataFrame(np.array([None, None, None, None, None, None, None, None]).reshape(-1,8),
                             columns=['entity','freq','response','proportion','first','last','mode','title'])
        # concatenate with previous dataframe by rows
        result = pd.concat([result,df])
    return(result)

In [29]:
# Training Data
train = pd.read_csv('/Users/jihunlee/Desktop/NER_Train.csv')
# Subset country only
train = train[~pd.isna(train.Country)&~pd.isna(train.Title)].reset_index()
train = train.iloc[range(200)].reset_index()

In [ ]:
train.Country.unique()

In [32]:
train_df = ner_dataframe(train)

# Machine Learning to Predict

In [41]:
train_df = train_df.dropna(how='any')

In [42]:
X, y = train_df[['proportion','first','last','mode','title']], train_df['response']

In [44]:
X = X.astype(float)

In [45]:
y = y.astype(float)

In [46]:
y = list(map(lambda x: 1 if x else 0, y))

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [48]:
X_train

,proportion,first,last,mode,title
2,0.500000,1.0,0.0,0.0,0.0
2,0.040000,0.0,0.0,0.0,0.0
3,0.045455,0.0,0.0,0.0,0.0
4,0.040000,0.0,0.0,0.0,0.0
0,0.006944,0.0,1.0,1.0,1.0
...,...,...,...,...,...
11,0.039801,0.0,0.0,0.0,0.0
3,0.012579,0.0,0.0,0.0,0.0
10,0.013889,0.0,0.0,0.0,0.0
16,0.010000,0.0,0.0,0.0,0.0


In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [1,3,5,7],
    'max_features': [3,4,5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [2,4,6,8],
    'n_estimators': [500, 1000, 1500]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)

In [50]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 10 folds for each of 432 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 3257 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed: 12.7min finished


{'bootstrap': True,
 'max_depth': 5,
 'max_features': 5,
 'min_samples_leaf': 3,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [51]:
from sklearn.metrics import accuracy_score
best_grid_rf = grid_search.best_estimator_
rf_predict = best_grid_rf.predict(X_test)
print(accuracy_score(y_test, rf_predict))
from sklearn.metrics import cohen_kappa_score
print(cohen_kappa_score(rf_predict, y_test, labels=None, weights=None, sample_weight=None))

0.9468354430379747
0.48545375597047336


In [ ]:
from xgboost import XGBClassifier

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [1,2,3, 4, 5]
        }

In [ ]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [ ]:
grid_search_xgb = GridSearchCV(estimator=xgb, param_grid=params, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3 )

In [ ]:
grid_search_xgb.fit(X_train, Y_train)

In [ ]:
grid_search_xgb.best_params_

In [ ]:
best_grid_xgb = grid_search_xgb.best_estimator_
xgb_predict = best_grid_xgb.predict(X_test)
accuracy_score(y_test, xgb_predict)

In [52]:
from sklearn.externals import joblib 
# Save the model as a pickle in a file 
joblib.dump(best_grid_rf, '/Users/jihunlee/Desktop/rf_location_extraction.pkl') 
# Load the model from the file 
rf_mod = joblib.load('/Users/jihunlee/Desktop/rf_location_extraction.pkl')  

In [ ]:
from sklearn.linear_model import LogisticRegression
X, y = train_df[['proportion','first','last','mode','title']], train_df['response']
clf = LogisticRegression(random_state=0).fit(X, y)
pred = clf.predict(X)
clf.predict_proba(X)
clf.score(X, y)


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gnb = GaussianNB()
y_pred = gnb.fit(X, y).predict(X)
accuracy_score(y_pred, y)


In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X,y)
y_pred2 = clf.predict(X)
accuracy_score(y,y_pred2)

In [ ]:
cohen_kappa_score(y_pred, y, labels=None, weights=None, sample_weight=None)

In [ ]:
from sklearn.metrics import cohen_kappa_score